In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")

In [3]:
import datajoint_utils as du
import datajoint as dj

INFO - 2021-02-02 08:45:19,808 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-02 08:45:19,809 - settings - Setting database.user to celiib
INFO - 2021-02-02 08:45:19,810 - settings - Setting database.password to newceliipass
INFO - 2021-02-02 08:45:19,818 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-02 08:45:19,818 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-02 08:45:19,831 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-02 08:45:20,113 - settings - Setting enable_python_native_blobs to True


In [4]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-02 08:45:20,189 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-02 08:45:20,483 - settings - Setting enable_python_native_blobs to True


# Inspecting Deimation

In [ ]:
# the keysource used
key_source = minnie.NucleiSegmentsRun2 - (minnie.Decimation() & dict(decimation_ratio=decimation_ratio_global)).proj()

In [ ]:
len(minnie.NucleiSegmentsRun2())

In [ ]:
len(minnie.Decimation() & "decimation_ratio=0.25" & minnie.NucleiSegmentsRun2())

# Inspecting Baylor Soma

In [ ]:
len(dj.U("segment_id") & minnie.BaylorSegmentCentroid())

In [ ]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000
current_version = 30

key_source =  (((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}") & (dj.U("segment_id") & minnie.NucleiSegmentsRun2()))
                                                                 
     
key_source

In [ ]:
segment_id = 864691135771738699

In [ ]:
minnie.NeuronGliaNuclei() & dict(segment_id=segment_id)

In [ ]:
minnie.BaylorSegmentCentroid()  & dict(segment_id=segment_id)

In [ ]:
minnie.Decimation() & key_source #& "n_faces>8000000"

In [ ]:
(curr_table & "key_hash='035e97d66228b147435ed67c87a7f29d'").fetch1("key")

In [ ]:
curr_table = (schema.jobs & "table_name='__baylor_segment_centroid'")
curr_table

# Checking multi-somas

In [ ]:
one_soma_segment_ids = (dj.U("segment_id") & (minnie.BaylorSegmentCentroid() & "multiplicity = 1")).fetch("segment_id")
one_soma_segment_ids

In [ ]:
n_faces = (minnie.BaylorSegmentCentroid() & "multiplicity = 1").fetch("n_faces")

In [ ]:
from matplotlib import pyplot as plt
_ = plt.hist(n_faces,bins=100)

In [ ]:
 small_soma_ids = (minnie.BaylorSegmentCentroid() & "multiplicity = 1" & "n_faces<20000").fetch("segment_id")

In [ ]:
du.plot_decimated_mesh_with_somas(small_soma_ids[-101])

In [ ]:
# Doing the bigger somas

big_soma_ids = (minnie.BaylorSegmentCentroid() & "multiplicity = 1" & "n_faces>60000").fetch("segment_id")
big_soma_ids.shape

In [ ]:
du.plot_decimated_mesh_with_somas(big_soma_ids[-4])

# Inspecting Decomposition

In [15]:
(minnie.Decomposition() & "n_somas>1")

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691134765143804,0,0.25,30.00,0,0,1,=BLOB=,44570,90882,0,1,1,1,2,1,1,1,28585.105732166514,28585.105732166514,28585.106,1726.047,1726.047,1726.047,1726.047,4,0.00013993301397863443,4.0,28585.106,1,0.00013993301266750595,4.0,174766472.25666714,13946453.78167041,6113.899801322207,6113.899744036882,174766472.25666714,89.0604
864691134768966140,0,0.25,30.00,5,0,1,=BLOB=,81637,164711,0,1,1,1,2,2,5,4,61206.675450776704,40401.85629631666,4713.508,108.165,108.165,1174.845,1135.231,2,3.2676174375921285e-05,0.4,6413.497,2,0.0003118423537112436,1.0,806371663.6001841,403185831.80009204,13174.570545800676,125730.41877156628,403185831.80009204,58.3114
864691134831878402,0,0.25,30.00,0,0,1,=BLOB=,141431,288006,0,1,0,1,2,3,5,3,167837.6994974118,105050.86635215634,42893.926,114.293,114.293,1035.029,1035.029,0,0.0,0.0,19892.907,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.6019
864691134874936141,0,0.25,30.00,0,0,1,=BLOB=,64451,129722,0,2,2,1,2,3,7,3,69751.43119438276,56515.80323426945,5610.567,124.571,124.571,1568.841,1568.841,4,5.7346493563018516e-05,0.5714285714285714,41954.196,3,9.534207257839e-05,1.333,123117238.4626452,18270225.275904972,1765.0854807486746,2934.5631712891172,41039079.4875484,143.5662
864691134884769018,0,0.25,30.00,0,0,1,=BLOB=,1528011,3078802,0,7,5,2,3,20,421,129,15358033.818095537,6359831.284514944,19558.966,117.674,117.674,462.331,428.373,3853,0.0002508784682750353,9.152019002375297,8180017.673,186,0.00047102587720778387,20.715,818307027581.6089,118226275.54461034,53282.01755991983,100037.31780221165,4399500148.288219,8443.518
864691134884772346,0,0.25,30.00,0,0,1,=BLOB=,385614,805879,0,1,1,1,2,10,106,51,3416069.9194424567,1568051.5996385978,22918.888,186.605,186.605,506.188,456.498,1063,0.0003111763005639809,10.028301886792454,2126475.354,56,0.0004998882296004264,18.982,207947661950.84503,109383813.78330484,60873.36233000306,97789.82933410713,37133511

In [22]:
len(minnie.Decomposition())

74696

In [14]:
import numpy as np
multiplicity = (minnie.Decomposition() & "n_somas>1").fetch("n_somas")
np.sum(multiplicity)

12647

In [13]:
multiplicity

array([1, 1, 1, ..., 1, 1, 1], dtype=object)

In [9]:
len(minnie.Decomposition())

74437

In [ ]:
curr_table = (schema.jobs & "table_name='__decomposition'")
curr_table & "status='error'"

In [ ]:
curr_table = (minnie.Decomposition() & "process_version = 5" & "n_limbs > 5" & dict(segment_id=864691134884800506))

In [ ]:
neuron_obj = curr_table.fetch1("decomposition")

In [ ]:
import neuron_visualizations as nviz

In [ ]:
nviz.visualize_neuron(neuron_obj,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all")

In [ ]:
nviz.plot_soma_limb_concept_network(neuron_obj)

In [ ]:
limb_results = pru.multi_soma_split_suggestions(neuron_obj)

In [ ]:
nviz.

In [ ]:
import proofreading_utils as pru
neuron_objs = pru.split_neuron(neuron_obj)

In [ ]:
n1,n2,n3 = neuron_objs

In [ ]:
nviz.visualize_neuron(n1,
                     limb_branch_dict="all")

In [ ]:
nviz.visualize_neuron(n3,
                     limb_branch_dict="all")

In [ ]:
len(minnie.Decomposition())

In [ ]:
minnie.Decomposition & "n_somas>4"

In [ ]:
np.unique((schema.jobs & "table_name='__decomposition'").fetch("error_message"))

In [ ]:
minnie.Decomposition() & "n_somas > 2"

# Inspecting the Multi-Soma Splitting

In [ ]:
minnie.Decomposition() & minnie.MultiSomaProofread2.proj()

# Deleting all the multi_soma splitting because made refinements

In [ ]:
(minnie.Decomposition() & "n_error_limbs>0 OR n_somas>1")

In [ ]:
(minnie.DecompositionSplit() & minnie.MultiSomaProofread2.proj())

# Inspecting Split Suggestions

In [ ]:
minnie,schema = du.configure_minnie_vm()

In [ ]:
split_table = schema.jobs & "table_name='__neuron_split_suggestions'"
split_table

# Decomposition Explaination

In [ ]:
minnie,schema = du.configure_minnie_vm()

In [ ]:
minnie.Decomposition()

In [ ]:
@schema
class DecompositionVersions(dj.Manual):
    definition="""
    # Descriptions of decomposition vertsion
    process_version          : int unsigned                 # key by which to lookup the decomposition process version
    ---
    description        : varchar(256)                  # new parts of the iteration of the decomposition process
    
    """

In [ ]:
DecompositionVersions()

In [ ]:
explainations_dict = [dict(process_version=0,description="1/15: First version"),
                      dict(process_version=1,description="1/18: Processed all Soma Containing Meshes"),
                      dict(process_version=2,description="1/20: Improved spine extraction"),
                      dict(process_version=3,description="1/20: Improved spine extraction")]
explainations_dict

In [ ]:
explainations_dict = [dict(process_version=4,description="1/25: Cleaned all_concept_starting_info")]
minnie.DecompositionVersions.insert(explainations_dict)

In [ ]:
DecompositionVersions

# Checking the Splitting For Allen

# 1/28 Segment Ids

In [ ]:
double_soma_1_28 = [864691136672813959,
864691136951653599,
864691135696109210,
864691136039382526,
864691135345140895,
864691135210077120,
864691134964412191,
864691136008578990,
864691135510515408,
864691135718447153,
864691135162240557,
864691135724299179,
864691136056568536,
864691135474614848,
864691135341171909,
864691136091147700,
864691137196942401,
864691135403709422,
864691135609476359,
864691135155711844,
864691135544409384,
864691135700441723,
864691136065225112,
864691135345291423,
864691136618586765,
864691135345134239,
864691137196949569,
864691135975576131,
864691136311815229,
864691135373423816,
864691136311803709,
864691136065514648,
864691136486808850,
864691136008511148,
864691136296681243,
864691135571307557,
864691135654120898,
864691136922900452,
864691135012398582,
864691136922748900,
864691136090951092,
864691136908675694,
                   ]
len(double_soma_1_28)

In [ ]:
len(minnie.Decomposition() & [dict(segment_id=k) for k in double_soma_1_28]

In [ ]:
triple_soma_1_28 = [864691136209271740,
864691135341203141,
864691135373436616,
864691135837279635,
864691135780852304,
864691135403794158,
864691136236789519,
864691135345280671,
864691136175165190,
864691136008595118,
864691135382504922,
864691135654121154,
864691136617159131,
864691135937310468,
864691135568932230,
864691135462292893,
864691135937302020,
864691136925391690,
864691137196946497,
864691135462454941,
864691135502074805,
864691135571346725,
864691135753746381]

len(triple_soma_1_28)

In [ ]:
minnie.Decomposition() & [dict(segment_id=k) for k in triple_soma_1_28]

In [ ]:
minnie,schema = du.configure_minnie_vm()

In [ ]:
key_source = minnie.DecompositionSplit() & (minnie.AllenProofreading() & dict(month=1,day=28,year=2021)).proj()
dj.U("segment_id") & key_source

In [ ]:
(schema.jobs & "table_name='__decomposition_split'" & "status='error'").delete()

In [ ]:
(schema.jobs & "table_name='__decomposition_split'" & "status='error'").fetch("key")

In [ ]:
print((schema.jobs & "table_name='__decomposition_split'" & "key_hash='1b66db350ea9742bfb564275904863c2'").fetch1("key"))

In [ ]:
import numpy as np
seg_ids_to_export = np.unique((minnie.DecompositionSplit() & (minnie.AllenProofreading() & dict(month=1,day=28,year=2021)).proj() ).fetch("segment_id"))
seg_ids_to_export.shape

In [ ]:
du.create_suggested_splits_neuroglancer_spreadsheet(segment_ids=seg_ids_to_export,
                                                   output_type="server",
                                                   output_filename="allen_spreadsheet_1_28.csv")